# Convert $A_s$ to $\sigma_8(z)$

Notebook to convert $A_s$ to $\sigma_8(z)$ and add that to the chain.

## Use `GetDist` to load chain(s) and $A_s$ from chain(s)

In [1]:
import getdist
from getdist import loadMCSamples

In [2]:
chain_settings={'ignore_rows':'0.'} # we don't want to remove any burn-in sample here
#input_chain_path=r'/nfs/turbo/lsa-nguyenmn/DESI_kp7/DESI_data/mock_blinded_y1/cobaya/base/desi-full-shape-power_bbn-omega_b/'
#input_chain_prefix=r'chain_desi-full-shape-power-taylor-velocileptors'
# see examples above to set the correct path to chain(s)
input_chain_path= # path to the directory that contains your chain
input_chain_prefix= # the prefix of your chain (minus the .xx.txt)
input_samples=loadMCSamples(input_chain_path+input_chain_prefix,settings=chain_settings)
params=input_samples.getParams()
h=params.H0/100.

In [3]:
import yaml

# Read the contents of the ASCII file
with open(input_chain_path+input_chain_prefix+'.input.yaml', 'r') as file:
    lines = file.readlines()

# Filter out problematic lines containing Python objects
filtered_lines = [line for line in lines if '!!python' not in line]

# Reconstruct the filtered content
filtered_data = ''.join(filtered_lines)

# Parse the modified content as YAML
parsed_data = yaml.safe_load(filtered_data)

# Access the value of 'N_ur'
N_ncdm = parsed_data.get('theory', {}).get('classy', {}).get('extra_args', {}).get('N_ncdm')
N_ur = parsed_data.get('theory', {}).get('classy', {}).get('extra_args', {}).get('N_ur')
m_ncdm = parsed_data.get('params', {}).get('m_ncdm')
tau_reio = parsed_data.get('params', {}).get('tau_reio')

## Define `CLASS` function to compute $\sigma_8$ from input cosmology

In [4]:
import classy
from classy import Class
import numpy as np

#settings to configure CLASS computations
classy_settings = {
    'output': 'mPk', #which quantities we want CLASS to compute
    'non linear' : 'none',
    
    'P_k_max_1/Mpc':50.0 # CLASS uses 1/Mpc units
}
A_s=np.exp(params.ln_A_s_1e10)/1E10 # Convert ln(1E10As) to As
# Below, we only need the values from the first sample in the chain as reference
cosmo_params = {
    'h': h[0],
    'omega_b': params.Omega_b[0]*h[0]**2, # classy uses omegabh2
    'omega_cdm': params.Omega_cdm[0]*h[0]**2, # classy uses omegach2
    'A_s': A_s[0],
    'n_s': params.n_s[0],
    'N_ncdm': N_ncdm,
    'N_ur': N_ur,
    'm_ncdm' : m_ncdm,
    #'recombination' : 'HyRec', # test if this changes your sigma8
    'tau_reio' : tau_reio,
}

def compute_sigma8z(settings,params,z=0.):
    #initialize Class instance and set parameters
    cosmo = Class();
    cosmo.set(settings);
    cosmo.set(params);
    cosmo.compute();
    return cosmo.sigma(8./cosmo.h(),z)

Check your `CLASS` and `classy` version. Some default settings might change between versions. So it's always a good idea to check...

In [5]:
print(f"CLASS version: {classy.__version__}")

CLASS version: v3.2.0


## Compute the reference $\sigma_8$

The rest of $\sigma_8$ values can just be linearly scaled.

In [6]:
sigma8z0_ref=compute_sigma8z(classy_settings,cosmo_params,z=0.)

In [7]:
sigma8z0_chain = sigma8z0_ref * np.sqrt(A_s/A_s[0])
sigma8z0_chain

array([0.80362203, 0.80362203, 0.80950524, ..., 0.78497506, 0.78497506,
       0.78497506])

## Add $\sigma_8$ and $S_8$ to the chain params

In [8]:
input_samples.addDerived(sigma8z0_chain,name='sigma_8',label='sigma_8')
input_samples.addDerived(sigma8z0_chain*np.sqrt(params.Omega_m/0.3),name='S_8',label='S_8')

## Save the new chains

In [10]:
# Output to the same directory, append `_sigma8` to the chain name
# Modify as you wish
output_chain = input_chain_path+input_chain_prefix+'_sigma8'
input_samples.saveAsText(output_chain)